[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/slitvinov/CUP3D/blob/AMR/fish.ipynb)

In [4]:
%%sh
git clone -q https://github.com/slitvinov/CUP3D.git
sudo apt-get install -qq libhdf5-mpich-dev libgsl-dev

fatal: destination path 'CUP3D' already exists and is not an empty directory.


In [5]:
%%sh
rm -f *.restart *.h5 *.xmf *.dat
OMPI_ALLOW_RUN_AS_ROOT=1 OMPI_ALLOW_RUN_AS_ROOT_CONFIRM=1 OMP_NUM_THREADS=1 \
exec mpiexec.mpich CUP3D/bin/main \
     -bMeanConstraint 2 \
     -bpdx 1 \
     -bpdy 1 \
     -bpdz 1 \
     -CFL 0.4 \
     -Ctol 0.1 \
     -extentx 1 \
     -factory-content \
     'StefanFish L=0.4 T=1.0 xpos=0.4 ypos=0.5 zpos=0.25 CorrectPosition=true CorrectZ=true CorrectRoll=true heightProfile=danio widthProfile=stefan bFixFrameOfRef=1
      StefanFish L=0.4 T=1.0 xpos=0.4 ypos=0.5 zpos=0.50 CorrectPosition=true CorrectZ=true CorrectRoll=true heightProfile=danio widthProfile=stefan
      StefanFish L=0.4 T=1.0 xpos=0.4 ypos=0.5 zpos=0.75 CorrectPosition=true CorrectZ=true CorrectRoll=true heightProfile=danio widthProfile=stefan' \
     -levelMax 4 \
     -levelStart 3 \
     -nu 0.001 \
     -poissonSolver iterative \
     -Rtol 5 \
     -tdump 0.5 \
     -tend 5 \

Cubism UP 3D (velocity-pressure 3D incompressible Navier-Stokes solver)
Running in RELEASE mode!
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
* Summary:
*    Parameter read from command line:                15
*    Parameter read from   0 file(s):                 0
*    Parameter read from defaults in code:            0
*    Total number of parameter read from all sources: 15
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
* Command Line:
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
CFL...............................................: 0.4
Ctol..............................................: 0.1
Rtol..............................................: 5
bMeanConstraint...................................: 2
bpdx..............................................: 1
bpdy..............................................: 1
bpdz..............................................: 1
extentx...........................................: 1
f

In [7]:
import glob
import h5py
import numpy as np
import sys
import os
import re
import xml.etree.ElementTree
import matplotlib.pyplot as plt
import matplotlib.animation

def plot(path):
    x = xml.etree.ElementTree.parse(path)
    root = x.getroot()
    hdf_data = root.find('.//DataItem[@Format="HDF"]')
    hdf_path = hdf_data.text
    hdf_path = re.sub("^[ \t\n]*", "", hdf_path)
    hdf_path = re.sub("[ \t\n]*$", "", hdf_path)
    hdf_data = re.sub("^.*:", "", hdf_path)
    hdf_path = re.sub(":[^:]*$", "", hdf_path)
    dirname = os.path.dirname(path)
    hdf_path = os.path.join(dirname, hdf_path)
    xx = [ ]
    yy = [ ]
    zz = [ ]
    with h5py.File(hdf_path) as f:
        offset = 0
        for grid in root.findall('.//Grid[@GridType="Uniform"]'):
            topology = grid.find('./Topology')
            geometry = grid.find('./Geometry[@GeometryType="ORIGIN_DXDYDZ"]')
            dorg, dspa = geometry.findall("./DataItem")
            ddim = topology.get("Dimensions")
            ox, oy, oz = [float(e) for e in dorg.text.split()]
            sx, sy, sz = [float(e) for e in dspa.text.split()]
            nx, ny, nz = [int(e) for e in ddim.split()]
            size = (nx - 1) * (ny - 1) * (nz - 1)
            data = f[hdf_data][offset:offset + size]
            data = data.reshape((nx - 1, ny - 1, nz - 1))
            ix, iy, iz = np.where(data > 0)
            x = (ix + 1 / 2) * sx + ox
            y = (iy + 1 / 2) * sy + oy
            z = (iz + 1 / 2) * sz + oz
            xx.append(x)
            yy.append(y)
            zz.append(z)
            offset += size
    points.set_data(np.concatenate(zz), np.concatenate(xx))
matplotlib.rc("animation", html="jshtml")
plt.ioff()
plt.axis((0, 1, 0, 1))
plt.axis("on")
plt.axis("equal")
points, = plt.plot([0, 1], [0, 1], 'o', alpha=0.1)
anim = matplotlib.animation.FuncAnimation(plt.gcf(), plot, glob.glob("chi_*.xmf"))
plt.close()
display(anim);